In [1]:
import chama
import pandas as pd
import numpy as np

# 1. simulations
类似于选择摄像头，求解最大监控范围类的问题，不需要做仿真( Note that if the sensor placement objective is to maximize geographic coverage, simulations are not required)

# 2. Sensor Technology 
类似于选择摄像头，求解最大监控范围类的问题，不需要对传感器做定义

#  3. Impact assessment

* 数据准备， 生成含有 **‘Sensor’and ‘Coverage’** 两列的 **DataFrame**，  **‘Sensor’** 为传感器编号， **‘Coverage’**为传感器可以监控的区域编号列表

The Coverage Formulation requires as input a DataFrame with two columns: ‘Sensor’, and ‘Coverage’, where the ‘Coverage’ is a list of entities covered by each sensor. The formulation optimizes a sensor layout that maximizes the coverage of the entities contained in this DataFrame. An entity to be covered might include scenarios, scenario-time pairs, or geographic locations.

In [2]:
# 读取摄像头信息
camera_monitor_info = np.load('./price_matrix_488_1000.npz')['list_array']
# # 生成摄像头ID
Sensors = np.arange(0, camera_monitor_info.shape[0])
# # 一共有100个区域， 随机生成每个摄像头所能看到的区域信息
# np.random.seed(10)
# Scenarios = np.random.randint(0,2,(10,100))
# # 生成DataFrame
# sensors = []
scenarios = []
for i in range(len(Sensors)):
    #sensor = Sensors[i]
    scenario = np.where(camera_monitor_info[i] == 1)[0].astype('str')
    #sensors.append(sensor)
    scenarios.append(list(scenario))

# # 生成监控区域表格, scenario coverage
scenario_cov_df = pd.DataFrame({"Sensor": Sensors, "Coverage":scenarios})
scenario_cov_df

,Sensor,Coverage
0,0,"[8372, 8373, 8417, 8418, 8457, 8528, 8529, 853..."
1,1,"[8359, 8388, 8389, 8390, 8391, 8392, 8393, 839..."
2,2,"[8363, 8400, 8401, 8402, 8403, 8404, 8405, 840..."
3,3,"[8323, 8324, 8325, 8326, 8327, 8328, 8329, 833..."
4,4,"[8263, 8264, 8265, 8266, 8267, 8294, 8295, 829..."
...,...,...
483,483,"[35, 47, 55, 56, 65, 66, 67, 68, 69, 78, 79, 8..."
484,484,"[29, 35, 36, 55, 56, 57, 58, 67, 68, 69, 70, 7..."
485,485,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
486,486,"[7905, 7906, 7907, 7908, 7947, 7948, 7949, 795..."


# 4. Optimization

In [3]:
coverageform = chama.optimize.CoverageFormulation()
results = coverageform.solve(coverage=scenario_cov_df, sensor_budget=100, use_sensor_cost=False)

In [4]:
np.array(results['Sensors'])

array([  3,   4,   6,   8,   9,  15,  17,  18,  19,  21,  27,  32,  34,
        35,  37,  49,  51,  52,  61,  66,  79,  80,  82,  84,  85,  89,
        93,  96,  99, 101, 108, 109, 111, 113, 117, 118, 121, 127, 141,
       143, 150, 153, 155, 162, 165, 175, 177, 178, 181, 184, 185, 188,
       197, 200, 210, 211, 219, 221, 222, 226, 232, 234, 246, 254, 255,
       257, 265, 269, 270, 279, 283, 284, 294, 305, 306, 314, 315, 317,
       319, 321, 324, 331, 342, 366, 395, 400, 402, 403, 412, 415, 457,
       462, 463, 474, 477, 478, 480, 482, 484, 486])

#  现有方式选点

In [5]:
A_matrix = np.load('./price_matrix_488_1000.npz')['list_array'].T  # 转置成列为供给点
camera_idx = np.arange(0, A_matrix.shape[1], 1)
choose_camera = np.zeros((488,))
cur_visible_area = np.zeros((A_matrix.shape[0], ))

for i in range(100):
    choose_idx = np.argmax(np.count_nonzero(A_matrix.T + cur_visible_area > 0, axis=1))
    cur_visible_area += A_matrix.T[choose_idx]
    choose_camera[choose_idx] = 1  # 选择该摄像头区域
    # 选过的点对应的可见区域赋值为0
    A_matrix[:, choose_idx] = 0

In [21]:
len(np.where(np.sum(camera_monitor_info[results['Sensors']], axis=0) > 0)[0])

7660

In [19]:
len(np.where(np.sum(camera_monitor_info[choose_camera == 1], axis = 0) > 0)[0])

7578

#  结果比对

In [6]:
print("摄像头总数488， 选择100个摄像头")
print("Chama cameras monitor area:{}".format(np.array(results['Objective'])))
print("Chama cameras selection:{}".format(np.array(results['Sensors'])))
print("===========================================")
print("Original cameras monitor area:{}".format(np.where(cur_visible_area > 0)[0].shape[0]))
print("Original cameras selection:{}".format(np.where(choose_camera == 1)[0]))

摄像头总数488， 选择100个摄像头
Chama cameras monitor area:7660.0
Chama cameras selection:[  3   4   6   8   9  15  17  18  19  21  27  32  34  35  37  49  51  52
  61  66  79  80  82  84  85  89  93  96  99 101 108 109 111 113 117 118
 121 127 141 143 150 153 155 162 165 175 177 178 181 184 185 188 197 200
 210 211 219 221 222 226 232 234 246 254 255 257 265 269 270 279 283 284
 294 305 306 314 315 317 319 321 324 331 342 366 395 400 402 403 412 415
 457 462 463 474 477 478 480 482 484 486]
Original cameras monitor area:7578
Original cameras selection:[  3   4   6   8   9  15  19  20  21  24  26  27  31  32  35  49  50  52
  61  66  80  81  84  85  88  89  91  99 101 108 109 111 113 117 118 121
 127 141 143 150 153 155 157 164 165 167 175 178 181 183 184 185 188 195
 197 200 211 218 219 221 222 245 246 249 258 260 269 270 275 279 284 291
 294 306 314 315 317 319 321 324 329 331 342 366 395 400 402 403 412 450
 452 453 462 473 474 477 480 482 484 486]
